# Test de plusieurs librairies pour résoudre des problèmes MILP

Test effectués sur Google or-tools, pulp, pyomo

In [1]:
import numpy as np
d = [80, 270, 250, 160, 180]  # customer demand
M = [500, 500, 500]            # factory capacity
I = 5                         # Customers
J = 3                             # Factories
cost = [[4, 6, 9],
        [5, 4, 7],
        [6, 3, 3],
        [8, 5, 3],
        [10, 8, 4],
       ]                                # transportation costs

## 1. Or-tools

In [1]:
from ortools.linear_solver import pywraplp

# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver("GLOP")
if not solver:
    print("Oh oh...")

# Create the variables x and y.
x = solver.NumVar(0, 1, "x")
y = solver.NumVar(0, 2, "y")

print("Number of variables =", solver.NumVariables())

# Create a linear constraint, 0 <= x + y <= 2.
ct = solver.Constraint(0, 2, "ct")
ct.SetCoefficient(x, 1)
ct.SetCoefficient(y, 1)

print("Number of constraints =", solver.NumConstraints())

# Create the objective function, 3 * x + y.
objective = solver.Objective()
objective.SetCoefficient(x, 3)
objective.SetCoefficient(y, 1)
objective.SetMaximization()

solver.Solve()

print("Solution:")
print("Objective value =", objective.Value())
print("x =", x.solution_value())
print("y =", y.solution_value())

Number of variables = 2
Number of constraints = 1
Solution:
Objective value = 4.0
x = 1.0
y = 1.0


In [2]:
#!/usr/bin/env python3
# Copyright 2010-2022 Google LLC
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""This model implements a sudoku solver."""

from ortools.sat.python import cp_model


def solve_sudoku():
    """Solves the sudoku problem with the CP-SAT solver."""
    # Create the model.
    model = cp_model.CpModel()

    cell_size = 3
    line_size = cell_size**2
    line = list(range(0, line_size))
    cell = list(range(0, cell_size))

    initial_grid = [
        [0, 6, 0, 0, 5, 0, 0, 2, 0],
        [0, 0, 0, 4, 0, 0, 0, 9, 0],
        [7, 0, 0, 6, 0, 0, 0, 1, 0],
        [0, 0, 6, 0, 3, 0, 4, 0, 0],
        [0, 0, 4, 0, 7, 0, 1, 0, 0],
        [0, 0, 5, 0, 9, 0, 8, 0, 0],
        [0, 4, 0, 0, 0, 1, 0, 0, 6],
        [0, 3, 0, 0, 0, 8, 0, 0, 0],
        [0, 2, 0, 0, 4, 0, 0, 5, 0],
    ]

    grid = {}
    for i in line:
        for j in line:
            grid[(i, j)] = model.NewIntVar(1, line_size, "grid %i %i" % (i, j))

    # AllDifferent on rows.
    for i in line:
        model.AddAllDifferent(grid[(i, j)] for j in line)

    # AllDifferent on columns.
    for j in line:
        model.AddAllDifferent(grid[(i, j)] for i in line)

    # AllDifferent on cells.
    for i in cell:
        for j in cell:
            one_cell = []
            for di in cell:
                for dj in cell:
                    one_cell.append(grid[(i * cell_size + di, j * cell_size + dj)])

            model.AddAllDifferent(one_cell)

    # Initial values.
    for i in line:
        for j in line:
            if initial_grid[i][j]:
                model.Add(grid[(i, j)] == initial_grid[i][j])

    # Solve and print out the solution.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status == cp_model.OPTIMAL:
        for i in line:
            print([int(solver.Value(grid[(i, j)])) for j in line])


solve_sudoku()

[4, 6, 1, 3, 5, 9, 7, 2, 8]
[8, 5, 2, 4, 1, 7, 6, 9, 3]
[7, 9, 3, 6, 8, 2, 5, 1, 4]
[2, 1, 6, 8, 3, 5, 4, 7, 9]
[9, 8, 4, 2, 7, 6, 1, 3, 5]
[3, 7, 5, 1, 9, 4, 8, 6, 2]
[5, 4, 7, 9, 2, 1, 3, 8, 6]
[1, 3, 9, 5, 6, 8, 2, 4, 7]
[6, 2, 8, 7, 4, 3, 9, 5, 1]


In [3]:
from ortools.linear_solver import pywraplp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["constraint_coeffs"] = [
        [5, 7, 9, 2, 1],
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -7],
    ]
    data["bounds"] = [250, 285, 211, 315]
    data["obj_coeffs"] = [7, 8, 2, 9, 6]
    data["num_vars"] = 5
    data["num_constraints"] = 4
    return data



def main():
    data = create_data_model()
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver("SCIP")
    if not solver:
        return

    infinity = solver.infinity()
    x = {}
    for j in range(data["num_vars"]):
        x[j] = solver.IntVar(0, infinity, "x[%i]" % j)
    print("Number of variables =", solver.NumVariables())

    for i in range(data["num_constraints"]):
        constraint = solver.RowConstraint(0, data["bounds"][i], "")
        for j in range(data["num_vars"]):
            constraint.SetCoefficient(x[j], data["constraint_coeffs"][i][j])
    print("Number of constraints =", solver.NumConstraints())
    # In Python, you can also set the constraints as follows.
    # for i in range(data['num_constraints']):
    #  constraint_expr = \
    # [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
    #  solver.Add(sum(constraint_expr) <= data['bounds'][i])

    objective = solver.Objective()
    for j in range(data["num_vars"]):
        objective.SetCoefficient(x[j], data["obj_coeffs"][j])
    objective.SetMaximization()
    # In Python, you can also set the objective as follows.
    # obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
    # solver.Maximize(solver.Sum(obj_expr))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Objective value =", solver.Objective().Value())
        for j in range(data["num_vars"]):
            print(x[j].name(), " = ", x[j].solution_value())
        print()
        print("Problem solved in %f milliseconds" % solver.wall_time())
        print("Problem solved in %d iterations" % solver.iterations())
        print("Problem solved in %d branch-and-bound nodes" % solver.nodes())
    else:
        print("The problem does not have an optimal solution.")


if __name__ == "__main__":
    main()

Number of variables = 5
Number of constraints = 4
Objective value = 259.99999999999966
x[0]  =  8.0
x[1]  =  21.0
x[2]  =  0.0
x[3]  =  2.0
x[4]  =  3.0

Problem solved in 92.000000 milliseconds
Problem solved in 71 iterations
Problem solved in 7 branch-and-bound nodes


In [62]:
import numpy as np
d = [80, 270, 250, 160, 180]  # customer demand
M = [500, 500, 500]            # factory capacity
I = 5                         # Customers
J = 3                             # Factories
cost = [[4, 6, 9],
        [5, 4, 7],
        [6, 3, 3],
        [8, 5, 3],
        [10, 8, 4],
       ]                                # transportation costs

In [80]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver("SCIP")
if not solver:
    print("Oh oh...")


# Création des variables.
x = [[0 for j in range(J)] for i in range(I)]
infinity = solver.infinity()
for i in range(I):
    for j in range(J):
        x[i][j] = solver.IntVar(0, infinity, f"x[{i}][{j}]")
print("Number of variables =", solver.NumVariables())

for i in range(I):
    acc = 0
    for j in range(J):
        acc += x[i][j]
    solver.Add(acc == d[i], f"Demand customer {i}")
for j in range(J):
    constraint = solver.RowConstraint(0, M[j], f"Factory {j} capacity")
    for i in range(I):
        constraint.SetCoefficient(x[i][j], 1)
print("Number of constraints =", solver.NumConstraints())

# In Python, you can also set the constraints as follows.
# for i in range(data['num_constraints']):
#  constraint_expr = \
# [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
#  solver.Add(sum(constraint_expr) <= data['bounds'][i])

objective = solver.Objective()
for i in range(I):
    for j in range(J):
        objective.SetCoefficient(x[i][j], cost[i][j])
objective.SetMinimization()

# In Python, you can also set the objective as follows.
# obj_expr = [cost[i][j] * x[i][j] for j in J for i in I]
# solver.Minimize(solver.Sum(obj_expr))

import time
start_time = time.time_ns()
status = solver.Solve()
end_time = time.time_ns()
print(f"Time taken : {(end_time - start_time) / 1e6} ms.")

if status == pywraplp.Solver.OPTIMAL:
    print("Objective value =", solver.Objective().Value())
    for i in range(I):
        for j in range(J):
            print(x[i][j].name(), " = ", x[i][j].solution_value())
    print()
    print("Problem solved in %f milliseconds" % solver.wall_time())
    print("Problem solved in %d iterations" % solver.iterations())
    print("Problem solved in %d branch-and-bound nodes" % solver.nodes())
else:
    print("The problem does not have an optimal solution.")

Number of variables = 15
Number of constraints = 8
Time taken : 4.2672 ms.
Objective value = 3349.9999999999995
x[0][0]  =  80.0
x[0][1]  =  0.0
x[0][2]  =  0.0
x[1][0]  =  0.0
x[1][1]  =  270.0
x[1][2]  =  0.0
x[2][0]  =  0.0
x[2][1]  =  90.0
x[2][2]  =  160.0
x[3][0]  =  0.0
x[3][1]  =  0.0
x[3][2]  =  160.0
x[4][0]  =  0.0
x[4][1]  =  0.0
x[4][2]  =  180.0

Problem solved in 20.000000 milliseconds
Problem solved in 6 iterations
Problem solved in 1 branch-and-bound nodes


## PulP

In [2]:
import sys
import numpy as np
d = {1:80, 2:270, 3:250, 4:160, 5:180}  # customer demand
M = {1:500, 2:500, 3:500}               # factory capacity
I = [1,2,3,4,5]                         # Customers
J = [1,2,3]                             # Factories
cost = {(1,1):4,    (1,2):6,    (1,3):9,
     (2,1):5,    (2,2):4,    (2,3):7,
     (3,1):6,    (3,2):3,    (3,3):3,
     (4,1):8,    (4,2):5,    (4,3):3,
     (5,1):10,   (5,2):8,    (5,3):4
   }    

In [26]:
import pulp
x = pulp.LpVariable.dicts("amount of goods", ((i, j) for i in I for j in J), lowBound = 0, cat = 'Continuous')


objective = pulp.LpAffineExpression(e = [(x[i,j],cost[i,j]) for i,j in x], name = 'Objective function')
model = pulp.LpProblem(name = "Transportation cost minimization", 
                        sense = pulp.LpMinimize)
model += pulp.lpSum(objective)


# Constraint: sum of goods == customer demand
for i in I:
    tmpExpression = pulp.LpAffineExpression(e = [(x[i,j], 1) for j in J if (i,j) in x])
    tmpConstraint = pulp.LpConstraint(e = pulp.lpSum(tmpExpression),
        sense = pulp.LpConstraintEQ,                                
        rhs = d[i])
    model.addConstraint(tmpConstraint)
# Constraint: sum of goods <= factory capacityy
for j in J:
    tmpExpression = pulp.LpAffineExpression(e = [(x[i,j], 1) for j in J if (i,j) in x])
    tmpConstraint = pulp.LpConstraint(e = pulp.lpSum(tmpExpression),
        sense = pulp.LpConstraintLE,
        rhs = M[j])
    model.addConstraint(tmpConstraint)


solver = pulp.get_solver('HiGHS_CMD', msg=True)
import time
start_time = time.time_ns()
results = model.solve(solver)
end_time = time.time_ns()
print(f"Time taken : {(end_time - start_time) / 1e6} ms.")

if model.status == 1:
    print('Solution is optimal: %s' %pulp.LpStatus[model.status])
else:
    print('Failed to find solution: %s' %pulp.LpStatus[model.status])
print('Objective function value =', pulp.value(model.objective))
EPS = 1.e-6
for (i,j) in x:
    if x[i,j].varValue > EPS:
        print("sending quantity %10s from factory %3s to customer %3s" % (x[i,j].varValue,j,i))

Time taken : 9.665499 ms.
Solution is optimal: Optimal
Objective function value = 3350.0
sending quantity       80.0 from factory   1 to customer   1
sending quantity      270.0 from factory   2 to customer   2
sending quantity      250.0 from factory   2 to customer   3
sending quantity      160.0 from factory   3 to customer   4
sending quantity      180.0 from factory   3 to customer   5


/home/leoku/mambaforge/envs/jugglyng/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [1]:
import pulp

print(pulp.list_solvers())
print(pulp.list_solvers(onlyAvailable=True))

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'XPRESS', 'XPRESS_PY', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD', 'HiGHS_CMD']
['GLPK_CMD', 'COIN_CMD', 'SCIP_CMD', 'HiGHS_CMD']


## Pyomo

In [85]:
import sys
import numpy as np
d = {1:80, 2:270, 3:250, 4:160, 5:180}  # customer demand
M = {1:500, 2:500, 3:500}               # factory capacity
I = [1,2,3,4,5]                         # Customers
J = [1,2,3]                             # Factories
cost = {(1,1):4,    (1,2):6,    (1,3):9,
     (2,1):5,    (2,2):4,    (2,3):7,
     (3,1):6,    (3,2):3,    (3,3):3,
     (4,1):8,    (4,2):5,    (4,3):3,
     (5,1):10,   (5,2):8,    (5,3):4
   }    

In [86]:
from pyomo import environ as pe
# ConcreteModel is model where data values supplied at the time of the model definition. As opposite to AbstractModel where data values are supplied in data file
model = pe.ConcreteModel()
# all iterables are to be converted into Set objects
model.d_cust_demand = pe.Set(initialize = d.keys())
model.M_fact_capacity = pe.Set(initialize = M.keys())
# Parameters
# Cartesian product of two sets creates list of tuples [((i1,j1),v1),((i2,j2),v2),...] !!!
model.transport_cost = pe.Param(
    model.d_cust_demand * model.M_fact_capacity,
    initialize = cost,
    within = pe.NonNegativeReals)
model.cust_demand = pe.Param(model.d_cust_demand, 
    initialize = d,
    within = pe.NonNegativeReals)
model.fact_capacity = pe.Param(model.M_fact_capacity, 
    initialize = M,
    within = pe.NonNegativeReals)


model.x = pe.Var(
    model.d_cust_demand * model.M_fact_capacity,
    domain = pe.NonNegativeReals,
    bounds = (0, max(d.values())))


model.objective = pe.Objective(
    expr = pe.summation(model.transport_cost, model.x),
    sense = pe.minimize)


# Constraints: sum of goods == customer demand
def meet_demand(model, customer):
    sum_of_goods_from_factories = sum(model.x[customer,factory] for factory in model.M_fact_capacity)
    customer_demand = model.cust_demand[customer]
    return sum_of_goods_from_factories == customer_demand
model.Constraint1 = pe.Constraint(model.d_cust_demand, rule = meet_demand)
# Constraints: sum of goods <= factory capacity
def meet_capacity(model, factory):
    sum_of_goods_for_customers = sum(model.x[customer,factory] for customer in model.d_cust_demand)
    factory_capacity = model.fact_capacity[factory]
    return sum_of_goods_for_customers <= factory_capacity
model.Constraint2 = pe.Constraint(model.M_fact_capacity, rule = meet_demand)


solver = pe.SolverFactory("glpk")
import time
start_time = time.time_ns()
solution = solver.solve(model)
end_time = time.time_ns()
print(f"Time taken : {(end_time - start_time) / 1e6} ms.")


from pyomo.opt import SolverStatus, TerminationCondition
if (solution.solver.status == SolverStatus.ok) and (solution.solver.termination_condition == TerminationCondition.optimal):
    print("Solution is feasible and optimal")
    print("Objective function value = ", model.objective())
elif solution.solver.termination_condition == TerminationCondition.infeasible:
    print ("Failed to find solution.")
else:
    # something else is wrong
    print(str(solution.solver))
assignments = model.x.get_values().items()
EPS = 1.e-6
for (customer,factory),x in sorted(assignments):
    if x > EPS:
        print("sending quantity %10s from factory %3s to customer %3s" % (x, factory, customer))

Time taken : 14.8669 ms.
Solution is feasible and optimal
Objective function value =  3350.0
sending quantity       80.0 from factory   1 to customer   1
sending quantity      270.0 from factory   2 to customer   2
sending quantity      250.0 from factory   3 to customer   3
sending quantity      160.0 from factory   3 to customer   4
sending quantity      180.0 from factory   3 to customer   5


In [ ]:
solver = pe.SolverFactory("cbc")
import time
start_time = time.time_ns()
solution = solver.solve(model)
end_time = time.time_ns()
print(f"Time taken : {(end_time - start_time) / 1e6} ms.")


from pyomo.opt import SolverStatus, TerminationCondition
if (solution.solver.status == SolverStatus.ok) and (solution.solver.termination_condition == TerminationCondition.optimal):
    print("Solution is feasible and optimal")
    print("Objective function value = ", model.objective())
elif solution.solver.termination_condition == TerminationCondition.infeasible:
    print ("Failed to find solution.")
else:
    # something else is wrong
    print(str(solution.solver))
assignments = model.x.get_values().items()
EPS = 1.e-6
for (customer,factory),x in sorted(assignments):
    if x > EPS:
        print("sending quantity %10s from factory %3s to customer %3s" % (x, factory, customer))

ERROR: Solver (cbc) returned non-zero return code (-11)


ApplicationError: Solver (cbc) did not exit normally

In [89]:
import pyomo.environ as pyo
from itertools import compress

pyomo_solvers_list = pyo.SolverFactory.__dict__['_cls'].keys()
solvers_filter = []
for s in pyomo_solvers_list:
    try:
        solvers_filter.append(pyo.SolverFactory(s).available())
    except:
        solvers_filter.append(False)
pyomo_solvers_list = list(compress(pyomo_solvers_list,solvers_filter))


ModuleNotFoundError: No module named 'gurobipy'
ModuleNotFoundError: No module named 'xpress'
ModuleNotFoundError: No module named 'gurobipy'
ModuleNotFoundError: No module named 'xpress'
ModuleNotFoundError: No module named 'xpress'
solver path
gjh


### Résultats des test :

| Model | Toy Example |
|---|---| 
|Or-tools(inmodel) | 20 ms |
|Or-tools(python) | 4 ms |
|Or-tools(gurobi, inmodel) | |
|Or-tools(gurobi, python) | |
|Pulp(GLPK) | 10 ms |
|Pulp(Coin) | |
|Pulp(Gurobi) | |
|Pyomo(GLPK) | 20 ms |
|Pyomo(Coin) | |
|Pyomo(Gurobi) | |

In [1]:
import pulp
import time
var, prob = pulp.LpProblem.fromMPS("MILP_problems/binkar10_1.mps")
print(pulp.list_solvers())
print(pulp.list_solvers(onlyAvailable=True))

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'XPRESS', 'XPRESS_PY', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD', 'HiGHS_CMD']
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2451486
Academic license 2451486 - for non-commercial use only - registered to le___@universite-paris-saclay.fr


/home/leoku/mambaforge/envs/jugglyng/lib/python3.9/site-packages/pulp/apis/gurobi_api.py:170: UserWarning: GUROBI error: Unknown parameter '_test'.
  warnings.warn("GUROBI error: {}.".format(e))


['GLPK_CMD', 'GUROBI_CMD', 'COIN_CMD', 'SCIP_CMD', 'HiGHS_CMD']


In [8]:
a = dict()
a[(1, 2)] = 3
a

{(1, 2): 3}

In [6]:
for solver in ['GUROBI_CMD', 'SCIP_CMD', 'HiGHS_CMD','GLPK_CMD']:
    print(f"\nTest avec {solver}") 
    solver = pulp.get_solver(solver)
    start_time = time.time_ns()
    results = prob.solve(solver)
    end_time = time.time_ns()
    print(f"Time taken : {(end_time - start_time) / 1e6} ms.")
    print(f"Solution is optimal : {pulp.LpStatus[prob.status]}")


Test avec GUROBI_CMD
Time taken : 4015.890294 ms.
Solution is optimal : Optimal

Test avec SCIP_CMD


KeyboardInterrupt: 

In [ ]:
import pulp
p = pulp.LpProblem(name = "ExactCover", sense = pulp.const.LpMaximize)